# Data Acquisition and Cleaning
<br>
Predictive modelling is highly dependent on the integrity of the data. For my part for the data acquisition, I first obtained data on the Oscar from 1927 -2020 from a reliable source on Kaggle as a starting point. Next I obtained the most up-to-date and reliable from TMDB using the TMDB API and also other useful data from the OMDB API which can be accessed using the IMDB ID which can be retrieve from the TMDB API
<br>
The aim of our project or more specifically our problem defination is to predict the winners of The 93rd Academy Awards which will be held later this month on 26th April 2021.
<br>
The first phase of the project is the Data Acquistion which will be dividied in a couple of steps:
<ol>1. Obtain Oscar Data from Kaggle and to scrape the data and only focus on the big 5 award categories.</ol>
<ol>2. Using the data collected, retrieve data from TMDB API.</ol>
<ol>3. Using the IMDB ID retrieved from the TMDB API, to collect more data such as total nomiations, total awards won, imdb rating, rotten 

In [3]:
# Importing neccesary libraries 
from tmdbv3api import TMDb, Movie, Discover, Genre
from tmdbv3api import Account
from tmdbv3api import Authentication
import numpy as np
import random
import pandas as pd
from collections import defaultdict
import requests as rq
import re
import traceback
import datetime
from collections import Counter
import omdb

In [4]:
##TMDB details
##Account details for getting movie reccomendations
##Run this block only once during a session
tmdb = TMDb()
tmdb.api_key = '0921b0cce35c0b2ec8b874614d1d0b47'
tmdb.language = 'en'

USERNAME = "cybercat94"
PASSWORD = "Wave1994@"
auth = Authentication(username=USERNAME, password=PASSWORD)
account = Account()
details = account.details()

print("You are logged in as %s. Your account ID is %s." % (details.username, details.id))
print("This session expires at: %s" % auth.expires_at)

You are logged in as cybercat94. Your account ID is 10198725.
This session expires at: 2021-04-22 12:16:28 UTC


## Scrapping and cleaning oscar_award_data.csv
We will start by scrapping the dataset to meet our needs and requirements.

In [3]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')
oscar_df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10390,2019,2020,92,WRITING (Original Screenplay),"Screenplay by Bong Joon Ho, Han Jin Won; Story...",Parasite,True
10391,2019,2020,92,JEAN HERSHOLT HUMANITARIAN AWARD,Geena Davis,NaN,True
10392,2019,2020,92,HONORARY AWARD,David Lynch,NaN,True
10393,2019,2020,92,HONORARY AWARD,Wes Studi,NaN,True


Renaming columns for better readabality and querying data from the year 1960 to 2020.

In [4]:
oscar_df= oscar_df.rename({'category':'oscar_category', 'name':'nominee', 'winner':'oscar_win'}, axis=1)
oscar_df.drop(['ceremony'], axis=1, inplace=True)
oscar_df = oscar_df.query('year_film >=1960')

Next is to get the total counts of nominations for each film.

In [5]:
no_of_wins_df = oscar_df[oscar_df['oscar_win']==True].groupby('film').size().reset_index(name='no_oscar_wins')
no_of_wins_df


,film,no_oscar_wins
0,'Round Midnight,1
1,12 Years a Slave,3
2,1917,3
3,20 Feet from Stardom,1
4,2001: A Space Odyssey,1
...,...,...
836,Z,2
837,Zero Dark Thirty,1
838,Zootopia,1
839,Zorba the Greek,3


In [6]:
no_of_nomination_df = oscar_df.groupby('film').size().reset_index(name='no_oscar_nominations')
df = pd.merge(no_of_wins_df, no_of_nomination_df, on=['film'], how='right').fillna(0)
pd.options.display.float_format = '{:,.0f}'.format
df

,film,no_oscar_wins,no_oscar_nominations
0,'38',0,1
1,'Crocodile' Dundee,0,1
2,'Round Midnight,1,2
3,(A) Torzija [(A) Torsion],0,1
4,...And Justice for All,0,2
...,...,...,...
3135,"sex, lies, and videotape",0,1
3136,the accountant,1,1
3137,the end,0,1
3138,Ådalen '31,0,1


For this project we will be focusing on major awards only, the so called "big 5".
<br>Best Actor
<br>Best Actress
<br>Best Director
<br>Best Picture
<br>Best Adapted Screenplay/Best Original Screenplay

<br>As the title of these awards kept changing through the years, we will have to filter through the data.


In [7]:
oscar_df = oscar_df.replace(to_replace={'oscar_category' : {'ACTOR IN A LEADING ROLE': 'Best Actor',
                                                            'ACTOR': 'Best Actor',
                                                            'ACTRESS': 'Best Actress',
                                                            'ACTRESS IN A LEADING ROLE': 'Best Actress',
                                                            'DIRECTING' : 'Best Director',
                                                            'BEST PICTURE' : 'Best Picture',
                                                            'BEST MOTION PICTURE' : 'Best Picture',
                                                            'WRITING (Adapted Screenplay)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Adapted from Other Material)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Based on Material from Another Medium)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay--based on material from another medium)' : 'Best Adapted Screenplay',
                                                            'WRITING (Screenplay Based on Material Previously Produced or Published)' : 'Best Adapted Screenplay',
                                                            'WRITING (Original Screenplay)' : 'Best Original Screenplay',
                                                            'WRITING (Screenplay Written Directly for the Screen)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--based on factual material or material not previously published or produced)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--written directly for the screen)' : 'Best Original Screenplay',
                                                            'WRITING (Screenplay Written Directly for the Screen--based on factual material or on story material not previously published or produced)' : 'Best Original Screenplay',
                                                            'WRITING (Story and Screenplay--based on material not previously published or produced)' : 'Best Original Screenplay'},}, value=None)

oscar_df = oscar_df.query('oscar_category in ["Best Actor", "Best Actress", "Best Director", "Best Picture", "Best Adapted Screenplay","Best Original Screenplay"]')
oscar_df = pd.merge(oscar_df, df, on=['film'], how='left')
oscar_df = oscar_df.reset_index()
oscar_df.drop(['index'], axis=1, inplace=True)
oscar_df

,year_film,year_ceremony,oscar_category,nominee,film,oscar_win,no_oscar_wins,no_oscar_nominations
0,1960,1961,Best Actor,Trevor Howard,Sons and Lovers,False,1,7
1,1960,1961,Best Actor,Burt Lancaster,Elmer Gantry,True,3,5
2,1960,1961,Best Actor,Jack Lemmon,The Apartment,False,5,10
3,1960,1961,Best Actor,Laurence Olivier,The Entertainer,False,0,1
4,1960,1961,Best Actor,Spencer Tracy,Inherit the Wind,False,0,4
...,...,...,...,...,...,...,...,...
1835,2019,2020,Best Original Screenplay,Written by Rian Johnson,Knives Out,False,0,1
1836,2019,2020,Best Original Screenplay,Written by Noah Baumbach,Marriage Story,False,1,6
1837,2019,2020,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,1917,False,3,10
1838,2019,2020,Best Original Screenplay,Written by Quentin Tarantino,Once upon a Time...in Hollywood,False,2,10


# TMDB API

## Get Movie Information from TMDB api
This function uses TMDB api(DISCOVER), to retrieve all the movies with the constraint year.
<br> It will retrieve basic details such as id, title, list of genre ids, vote average and lastly vote count.
<br> We will later retrieve more information using the getDetails function to get details like casts, crews and etc.

In [15]:
def get_movies_by_year(year):

    movie = Movie()

    # initialiZe empty dictionary
    _movie_dict = {"tmdb_id":[], 
                   "year":[], 
                   "title":[], 
                   "genre":[], 
                   "tmdb_vote_average":[], 
                   "tmdb_vote_count":[]};
    
    print(f"In {year}")
    
    for i in range(1,501):
        discover = Discover()
        movie = discover.discover_movies({
            'year': year,
            'page': i,
            'certification_country': 'US'
        })
        
        for m in movie:
            _movie_dict["tmdb_id"].append(m.id)
            _movie_dict["year"].append(year)
            _movie_dict["title"].append(m.title)
            _movie_dict["genre"].append(m.title)
            _movie_dict["tmdb_vote_average"].append(m.title)
            _movie_dict["tmdb_vote_count"].append(m.title)

        if i == int(discover.total_pages):
            break

    return _movie_dict

Function to translate the genre id

In [16]:
def translate_genre_id(g):
    for i in range(0,len(g)):
        if g[i] == 28:
            g[i] = 'Action'
        elif g[i] == 12:
            g[i] = 'Adventure'
        elif g[i] == 16:
            g[i] = 'Animation'
        elif g[i] == 35:
            g[i] = 'Comedy'
        elif g[i] == 80:
            g[i] = 'Crime'
        elif g[i] == 99:
            g[i] = 'Documentary'
        elif g[i] == 18:
            g[i] = 'Drama'
        elif g[i] == 10751:
            g[i] = 'Family'
        elif g[i] == 14:
            g[i] = 'Fantasy'
        elif g[i] == 36:
            g[i] = 'History'
        elif g[i] == 27:
            g[i] = 'Horror'
        elif g[i] == 10402:
            g[i] = 'Music'
        elif g[i] == 9648:
            g[i] = 'Mystery'
        elif g[i] == 10749:
            g[i] = 'Romance'
        elif g[i] == 878:
            g[i] = 'Science Fiction'
        elif g[i] == 10770:
            g[i] = 'TV Movie'
        elif g[i] == 53:
            g[i] = 'Thriller'
        elif g[i] == 10752:
            g[i] = 'War'
        elif g[i] == 37:
            g[i] = 'Western'
    return g

Function to retrieve movie details with movie name and year

In [17]:
def search_movie_by_name_year(details_dict):
    
    oscar_dict = {"tmdb_id":[], 
                  "year_film":[], 
                  "year_ceremony":[], 
                  "film":[], 
                  "oscar_category":[], 
                  "nominee":[],
                  "no_oscar_wins":[], 
                  "no_oscar_nominations":[], 
                  "oscar_win":[],
                  "genre_id":[], 
                  "tmdb_vote_average":[],
                  "tmdb_vote_count":[]};
    
    movie = Movie()
    for item in details_dict:
        s = movie.search(item["film"])
        size = len(s)
        for i in range(0, size):
            result = s[i]
            try:
                if (result.release_date[0:4] == str(item["year_film"]) or result.release_date[0:4] == str(item["year_film"]-1) or result.release_date[0:4] == str(item["year_film"]-2)):
                    oscar_dict["tmdb_id"].append(result.id)
                    oscar_dict["year_film"].append(item["year_film"])
                    oscar_dict["year_ceremony"].append(item["year_ceremony"])
                    oscar_dict["oscar_category"].append(item["oscar_category"])
                    oscar_dict["film"].append(item["film"])
                    oscar_dict["nominee"].append(item["nominee"])
                    oscar_dict["no_oscar_wins"].append(item["no_oscar_wins"])
                    oscar_dict["no_oscar_nominations"].append(item["no_oscar_nominations"])
                    oscar_dict["oscar_win"].append(item["oscar_win"])
                    oscar_dict["genre_id"].append(result.genre_ids)
                    oscar_dict["tmdb_vote_average"].append(int((result.vote_average)*10))
                    oscar_dict["tmdb_vote_count"].append(result.vote_count)
                    break
            except NameError:
                print("no release date")
            except:
                print("Error can't find movie")

    return oscar_dict

In [242]:
details_dict = oscar_df.to_dict('r')
df = pd.DataFrame(search_movie_by_name_year(details_dict))

Error can't find movie
Error can't find movie


In [243]:
df

,tmdb_id,year_film,year_ceremony,film,oscar_category,nominee,no_oscar_wins,no_oscar_nominations,oscar_win,genre_id,tmdb_vote_average,tmdb_vote_count
0,53939,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,1,7,False,[18],69,18
1,22013,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,3,5,True,[18],72,85
2,284,1960,1961,The Apartment,Best Actor,Jack Lemmon,5,10,False,"[35, 18, 10749]",82,1429
3,18929,1960,1961,The Entertainer,Best Actor,Laurence Olivier,0,1,False,"[18, 10402]",62,28
4,1908,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,0,4,False,"[18, 36]",78,252
...,...,...,...,...,...,...,...,...,...,...,...,...
1833,546554,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,0,1,False,"[35, 80, 18, 9648]",78,7011
1834,492188,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,1,6,False,[18],78,4745
1835,530915,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,3,10,False,"[10752, 18, 28, 53]",79,7950
1836,466272,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,2,10,False,"[35, 18, 53]",75,8462


Function to retrieve top 4 casts 

In [18]:
def get_top_4_casts(creds):
    
    cast = (creds["cast"])
    cast_list = []
    
    for i in range(0,4):
        try:
            cast_list.append(cast[i].name)
        except:
            print("OUT OF RANGE")
        
    return cast_list

Function to retrieve total popularity score of top 4 casts 

In [19]:
def get_top_4_casts_popularity_score(creds):
    
    cast = (creds["cast"])
    popularity_score = 0

    for i in range(0,4):
        try:
            popularity_score += cast[i].popularity
        except:
            print("OUT OF RANGE")
        
    return popularity_score

Function to get total casts score

In [20]:
def get_casts_popularity_score(creds):
    
    cast = (creds["cast"])
    popularity_score = 0

    for c in cast:
        try:
            popularity_score += c.popularity
        except:
            print("OUT OF RANGE")
        
    return popularity_score

Function to retrieve crew based on positions

In [21]:
def get_crew_position(creds, job_id):
    crew = (creds["crew"])
    producer_list = []
    screenplay_list = []
    for c in crew:
        try:
            if c.job == 'Director' and job_id == 0:
                director = c.name
            elif c.job == 'Producer'and job_id == 1:
                producer_list.append(c.name)
            elif c.job == 'Screenplay'and job_id == 2:
                screenplay_list.append(c.name)
        except:
             print("OUT OF RANGE")
    if job_id == 0:
        return director
    elif job_id == 1:
        return producer_list
    elif job_id == 2:
        return screenplay_list

Function to get crew score

In [22]:
def get_crew_score(creds):
    crew = (creds["crew"])
    popularity_score = 0
    for c in crew:
        try:
            popularity_score += c.popularity
        except:
            print("OUT OF RANGE")
        
    return popularity_score

Function to get more movie details

In [249]:
def get_movie_details(oscar_dict):
    
    movie = Movie()
    
    movie_detials_dict = {"tmdb_id":[],
                           "imdb_id":[],
                           "year_film":[],
                           "year_ceremony":[], 
                           "film":[], 
                           "oscar_category":[], 
                           "nominee":[],
                           "oscar_win":[],
                           "no_oscar_wins":[], 
                           "no_oscar_nominations":[], 
                           "genre_id":[],
                           "staring_casts":[],
                           "casts_popularity":[],
                           "director":[],
                           "producers":[],
                           "screenplay":[],
                           "crew_popularity":[],
                           "budget":[],
                           "revenue":[],
                           "runtime":[],
                           "tmdb_vote_average":[], 
                           "tmdb_vote_count":[]};
                        
    for item in oscar_dict:
        if item["tmdb_id"] != 0:
            m = movie.details(item["tmdb_id"])
            movie_detials_dict["tmdb_id"].append(item["tmdb_id"])
            movie_detials_dict["imdb_id"].append(m.imdb_id)
            movie_detials_dict["year_film"].append(item["year_film"])
            movie_detials_dict["year_ceremony"].append(item["year_ceremony"])
            movie_detials_dict["film"].append(item["film"])
            movie_detials_dict["oscar_category"].append(item["oscar_category"])
            movie_detials_dict["nominee"].append(item["nominee"])
            movie_detials_dict["oscar_win"].append(item["oscar_win"])
            movie_detials_dict["no_oscar_wins"].append(item["no_oscar_wins"])
            movie_detials_dict["no_oscar_nominations"].append(item["no_oscar_nominations"])
            movie_detials_dict["genre_id"].append(item["genre_id"])
            movie_detials_dict["staring_casts"].append(get_top_4_casts(m.casts))
            movie_detials_dict["casts_popularity"].append(get_casts_popularity_score(m.casts))
            movie_detials_dict["director"].append(get_crew_position(m.casts, 0))
            movie_detials_dict["producers"].append(get_crew_position(m.casts, 1))
            movie_detials_dict["screenplay"].append(get_crew_position(m.casts, 2))
            movie_detials_dict["crew_popularity"].append(get_crew_score(m.casts))
            movie_detials_dict["budget"].append(m.budget)
            movie_detials_dict["revenue"].append(m.revenue)
            movie_detials_dict["runtime"].append(m.runtime)
            movie_detials_dict["tmdb_vote_average"].append(item["tmdb_vote_average"])
            movie_detials_dict["tmdb_vote_count"].append(item["tmdb_vote_count"])
                          
    return movie_detials_dict


In [250]:
oscar_dict = df.to_dict('r')
final_df = pd.DataFrame(get_movie_details(oscar_dict))

/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


OUT OF RANGE
OUT OF RANGE
OUT OF RANGE
OUT OF RANGE
OUT OF RANGE


In [251]:
final_df

,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_category,nominee,oscar_win,no_oscar_wins,no_oscar_nominations,...,casts_popularity,director,producers,screenplay,crew_popularity,budget,revenue,runtime,tmdb_vote_average,tmdb_vote_count
0,53939,tt0054326,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,False,1,7,...,21,Jack Cardiff,[],[Gavin Lambert],6,0,0,103,69,18
1,22013,tt0053793,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,True,3,5,...,39,Richard Brooks,[Bernard Smith],[Richard Brooks],15,3000000,10400000,146,72,85
2,284,tt0053604,1960,1961,The Apartment,Best Actor,Jack Lemmon,False,5,10,...,42,Billy Wilder,[Billy Wilder],[],40,3000000,25000000,125,82,1429
3,18929,tt0053796,1960,1961,The Entertainer,Best Actor,Laurence Olivier,False,0,1,...,38,Tony Richardson,[Harry Saltzman],"[John Osborne, Nigel Kneale]",32,0,0,96,62,28
4,1908,tt0053946,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,False,0,4,...,73,Stanley Kramer,"[Stanley Kramer, Herman Shumlin]","[Nedrick Young, Harold Jacob Smith]",21,0,0,128,78,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,546554,tt8946378,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,False,0,1,...,140,Rian Johnson,"[Rian Johnson, Ram Bergman]",[],70,40000000,309232797,131,78,7011
1834,492188,tt7653254,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,False,1,6,...,143,Noah Baumbach,"[Noah Baumbach, David Heyman]",[],82,19000000,2300000,137,78,4745
1835,530915,tt8579674,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,False,3,10,...,96,Sam Mendes,"[Sam Mendes, Callum McDougall, Pippa Harris, J...","[Sam Mendes, Krysty Wilson-Cairns]",161,100000000,374733942,119,79,7950
1836,466272,tt7131622,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,False,2,10,...,299,Quentin Tarantino,"[Quentin Tarantino, David Heyman, Shannon McIn...",[Quentin Tarantino],226,95000000,374251247,162,75,8462


# OMDB API
Next, we would like to get more information on the movie using the IMDB ID that we obtained using the TMDB API.
<br>We want to get information like:
<ol>1. Metascore </ol>
<ol>2. Imdb score </ol>
<ol>3. Rotten score </ol>
<ol>4. other nominations and wins </ol>


In [5]:
omdb.set_default('apikey', '4a7f4869')

In [6]:
def get_omdb_details(oscar_dict):
    
    omdb_detials_dict = {"imdb_id":[],
                         "oscar_category":[],
                         "nominee":[],
                         "imdb_rating":[],
                         "imdb_votes":[],
                         "rotten_tomatoes_rating":[],
                         "metascore":[],
                         "total_wins":[],
                         "total_nominations":[]};
    for item in oscar_dict:
        try:
            c = omdb.imdbid(item['imdb_id'])
            words = (c['awards'].split())
            imdb_rating = int((c['imdb_rating']).replace(".",""))
            imdb_votes = int((c['imdb_votes']).replace(',',''))
            if len(c['ratings']) > 1:
                rotten_tomatoes_rating = int((c['ratings'][1]['value'])[0:2])
            else: 
                rotten_tomatoes_rating = "N/A"
            if c['metascore'] != "N/A":
                metascore = int(c['metascore'])
            else:
                metascore = c['metascore']
            for key in words:
                try:
                    if key == "wins":
                        total_wins = int(words[(words.index(key)-1)])
                    if key == "nominations.":
                        total_nominations = int(words[(words.index(key)-1)])
                except:
                    print("not found")
            omdb_detials_dict["imdb_id"].append(item['imdb_id'])
            omdb_detials_dict["oscar_category"].append(item['oscar_category'])
            omdb_detials_dict["nominee"].append(item['nominee'])
            omdb_detials_dict["imdb_rating"].append((imdb_rating))
            omdb_detials_dict["imdb_votes"].append((imdb_votes))
            omdb_detials_dict["rotten_tomatoes_rating"].append((rotten_tomatoes_rating))
            omdb_detials_dict["metascore"].append((metascore))
            omdb_detials_dict["total_wins"].append(int(total_wins))
            omdb_detials_dict["total_nominations"].append(int(total_nominations))
        except:
            print("error")
            print(c['title'])
    return omdb_detials_dict

In [7]:
def get_omdb_nominated(oscar_dict):
    
    omdb_detials_dict = {"imdb_id":[],
                         "oscar_category":[],
                         "nominee":[],
                         "imdb_rating":[],
                         "imdb_votes":[],
                         "revenue":[],
                         "rotten_tomatoes_rating":[],
                         "metascore":[],
                         "total_wins":[],
                         "no_oscar_nominations":[],
                         "total_nominations":[]};
    for item in oscar_dict:
        try:
            c = omdb.imdbid(item['imdb_id'])
            words = (c['awards'].split())
            imdb_rating = int((c['imdb_rating']).replace(".",""))
            imdb_votes = int((c['imdb_votes']).replace(',',''))
            if len(c['ratings']) > 1:
                rotten_tomatoes_rating = int((c['ratings'][1]['value'])[0:2])
            else: 
                rotten_tomatoes_rating = "N/A"
            if c['metascore'] != "N/A":
                metascore = int(c['metascore'])
            else:
                metascore = c['metascore']
            for key in words:
                try:
                    if key == "wins":
                        total_wins = int(words[(words.index(key)-1)])
                    if key == "nominations.":
                        total_nominations = int(words[(words.index(key)-1)])
                    if key == "Oscars.":
                        no_oscar_nominations = int(words[(words.index(key)-1)])
                except:
                    print("not found")
            if(item['revenue']==0): 
                if(c['box_office']!='N/A'):
                    box = int((c['box_office']).replace("$", "").replace(",", ""));
                    omdb_detials_dict["revenue"].append(box)
                else:
                    omdb_detials_dict["revenue"].append(item['revenue'])
                    print("not found")
            else:
                omdb_detials_dict["revenue"].append(item['revenue'])
                
        
            if(item['no_oscar_nominations']==0): 
                omdb_detials_dict["no_oscar_nominations"].append(no_oscar_nominations)
            else:
                omdb_detials_dict["no_oscar_nominations"].append(item['no_oscar_nominations'])
                
            omdb_detials_dict["imdb_id"].append(item['imdb_id'])
            omdb_detials_dict["oscar_category"].append(item['oscar_category'])
            omdb_detials_dict["nominee"].append(item['nominee'])
            omdb_detials_dict["imdb_rating"].append((imdb_rating))
            omdb_detials_dict["imdb_votes"].append((imdb_votes))
            omdb_detials_dict["rotten_tomatoes_rating"].append((rotten_tomatoes_rating))
            omdb_detials_dict["metascore"].append((metascore))
            omdb_detials_dict["total_wins"].append(int(total_wins))
            omdb_detials_dict["total_nominations"].append(int(total_nominations))
        except:
            print("error")
            print(c['title'])
    return omdb_detials_dict

In [254]:
imdb = oscar_details.to_dict('r')
omdb_df = pd.DataFrame(get_omdb_details(imdb))

/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [255]:
omdb_df

,imdb_id,oscar_category,nominee,imdb_rating,imdb_votes,rotten_tomatoes_rating,metascore,total_wins,total_nominations
0,tt0054326,Best Actor,Trevor Howard,71,1408,67,N/A,7,16
1,tt0053793,Best Actor,Burt Lancaster,78,10534,94,N/A,8,13
2,tt0053604,Best Actor,Jack Lemmon,83,165873,93,94,19,8
3,tt0053796,Best Actor,Laurence Olivier,71,3022,77,70,19,4
4,tt0053946,Best Actor,Spencer Tracy,81,27478,92,75,3,7
...,...,...,...,...,...,...,...,...,...
1833,tt8946378,Best Original Screenplay,Written by Rian Johnson,79,471083,97,82,47,107
1834,tt7653254,Best Original Screenplay,Written by Noah Baumbach,79,253928,94,94,126,269
1835,tt8579674,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,83,442036,89,78,127,198
1836,tt7131622,Best Original Screenplay,Written by Quentin Tarantino,76,563506,85,83,134,372


In [256]:
final_movie_df = pd.merge(final_df, omdb_df, how='left', on=["imdb_id", "oscar_category", "nominee"])
final_movie_df

,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_category,nominee,oscar_win,no_oscar_wins,no_oscar_nominations,...,revenue,runtime,tmdb_vote_average,tmdb_vote_count,imdb_rating,imdb_votes,rotten_tomatoes_rating,metascore,total_wins,total_nominations
0,53939,tt0054326,1960,1961,Sons and Lovers,Best Actor,Trevor Howard,False,1,7,...,0,103,69,18,71,1408,67,N/A,7,16
1,22013,tt0053793,1960,1961,Elmer Gantry,Best Actor,Burt Lancaster,True,3,5,...,10400000,146,72,85,78,10534,94,N/A,8,13
2,284,tt0053604,1960,1961,The Apartment,Best Actor,Jack Lemmon,False,5,10,...,25000000,125,82,1429,83,165873,93,94,19,8
3,18929,tt0053796,1960,1961,The Entertainer,Best Actor,Laurence Olivier,False,0,1,...,0,96,62,28,71,3022,77,70,19,4
4,1908,tt0053946,1960,1961,Inherit the Wind,Best Actor,Spencer Tracy,False,0,4,...,0,128,78,252,81,27478,92,75,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,546554,tt8946378,2019,2020,Knives Out,Best Original Screenplay,Written by Rian Johnson,False,0,1,...,309232797,131,78,7011,79,471083,97,82,47,107
1834,492188,tt7653254,2019,2020,Marriage Story,Best Original Screenplay,Written by Noah Baumbach,False,1,6,...,2300000,137,78,4745,79,253928,94,94,126,269
1835,530915,tt8579674,2019,2020,1917,Best Original Screenplay,Written by Sam Mendes & Krysty Wilson-Cairns,False,3,10,...,374733942,119,79,7950,83,442036,89,78,127,198
1836,466272,tt7131622,2019,2020,Once upon a Time...in Hollywood,Best Original Screenplay,Written by Quentin Tarantino,False,2,10,...,374251247,162,75,8462,76,563506,85,83,134,372


In [257]:
db_path='Final_Movie_Data''.csv'
final_movie_df.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: Final_Movie_Data.csv


# Prepare the Result Dataset Using BeautifulSoup, TMBD API, OMDB API
For this portion we will be using Beautiful Soup for webscrapping the nomiated movies for 2021.
We will create separate dataset for Best Picture, Best Director, Best Actor, Best Actress, Best Original Screenplay and Best Adapted Screenplay

In [1]:
import requests
from bs4 import BeautifulSoup

### Best Picture

In [8]:
wiki_url = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture'
table_name = 'wikitable' 
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')
soup_table = soup.find_all('table', {'class':table_name})
print(len(soup_table))
df = pd.read_html(str(soup_table))
df

12


[              Year                 Film             Film Studio
 0     1927/28(1st)                  NaN                     NaN
 1     1927/28(1st)                Wings    Famous Players-Lasky
 2     1927/28(1st)           7th Heaven                     Fox
 3     1927/28(1st)           The Racket       The Caddo Company
 4  1928/29(2nd)[a]                  NaN                     NaN
 5  1928/29(2nd)[a]  The Broadway Melody     Metro-Goldwyn-Mayer
 6  1928/29(2nd)[a]                Alibi     Feature Productions
 7  1928/29(2nd)[a]      Hollywood Revue     Metro-Goldwyn-Mayer
 8  1928/29(2nd)[a]       In Old Arizona                     Fox
 9  1928/29(2nd)[a]          The Patriot  Paramount Famous Lasky,
              Year                            Film  \
 0    1929/30(3rd)                             NaN   
 1    1929/30(3rd)  All Quiet on the Western Front   
 2    1929/30(3rd)                   The Big House   
 3    1929/30(3rd)                        Disraeli   
 4    1929/30(

We only want the 3rd last table

In [9]:
df = df[10]
df

,Year,Film,Producer(s)
0,2020/21 (93rd),The Father,"David Parfitt, Jean-Louis Livi, and Philippe C..."
1,2020/21 (93rd),Judas and the Black Messiah,"Shaka King, Charles D. King, and Ryan Coogler"
2,2020/21 (93rd),Mank,"Ceán Chaffin, Eric Roth, and Douglas Urbanski"
3,2020/21 (93rd),Minari,Christina Oh
4,2020/21 (93rd),Nomadland,"Frances McDormand, Peter Spears, Mollye Asher,..."
5,2020/21 (93rd),Promising Young Woman,"Ben Browning, Ashley Fox, Emerald Fennell, and..."
6,2020/21 (93rd),Sound of Metal,Bert Hamelinck and Sacha Ben Harroche
7,2020/21 (93rd),The Trial of the Chicago 7,Marc Platt and Stuart M. Besser


In [10]:
category = ['Best Picture', 'Best Picture', 'Best Picture', 'Best Picture', 'Best Picture', 'Best Picture', 'Best Picture', 'Best Picture']
df['oscar_category'] = category
df.Year = 2020
df['year_ceremony'] = 2021
df['no_oscar_nominations'] = 0;
df['oscar_win'] = False;
df = df.rename({'Producer(s)':'nominee', 'Year':'year_film', 'Film':'film'}, axis=1)
df

,year_film,film,nominee,oscar_category,year_ceremony,no_oscar_nominations,oscar_win
0,2020,The Father,"David Parfitt, Jean-Louis Livi, and Philippe C...",Best Picture,2021,0,False
1,2020,Judas and the Black Messiah,"Shaka King, Charles D. King, and Ryan Coogler",Best Picture,2021,0,False
2,2020,Mank,"Ceán Chaffin, Eric Roth, and Douglas Urbanski",Best Picture,2021,0,False
3,2020,Minari,Christina Oh,Best Picture,2021,0,False
4,2020,Nomadland,"Frances McDormand, Peter Spears, Mollye Asher,...",Best Picture,2021,0,False
5,2020,Promising Young Woman,"Ben Browning, Ashley Fox, Emerald Fennell, and...",Best Picture,2021,0,False
6,2020,Sound of Metal,Bert Hamelinck and Sacha Ben Harroche,Best Picture,2021,0,False
7,2020,The Trial of the Chicago 7,Marc Platt and Stuart M. Besser,Best Picture,2021,0,False


Use TMDB API and OMDB API to get data we want

In [11]:
def search_nom_by_name_year(dict):
    
    nominee_dict = {"tmdb_id":[], 
                  "year_film":[], 
                  "year_ceremony":[],
                  "oscar_win":[],
                  "no_oscar_nominations":[], 
                  "film":[], 
                  "oscar_category":[], 
                  "nominee":[],
                  "genre_id":[], 
                  "tmdb_vote_average":[],
                  "tmdb_vote_count":[]};
    
    movie = Movie()
    for item in dict:
        s = movie.search(item["film"])
        size = len(s)
        for i in range(0, size):
            result = s[i]
            try:
                if (result.release_date[0:4] == str(item["year_film"]) or result.release_date[0:4] == str(item["year_film"]+1)):
                    nominee_dict["tmdb_id"].append(result.id)
                    nominee_dict["year_film"].append(item["year_film"])
                    nominee_dict["year_ceremony"].append(item["year_ceremony"])
                    nominee_dict["oscar_win"].append(item["oscar_win"]) 
                    nominee_dict["no_oscar_nominations"].append(item["no_oscar_nominations"]) 
                    nominee_dict["film"].append(item["film"])
                    nominee_dict["oscar_category"].append(item["oscar_category"])
                    nominee_dict["nominee"].append(item["nominee"])
                    nominee_dict["genre_id"].append(result.genre_ids)
                    nominee_dict["tmdb_vote_average"].append(int((result.vote_average)*10))
                    nominee_dict["tmdb_vote_count"].append(result.vote_count)
                    break
            except NameError:
                print("no release date")
            except:
                print("Error can't find movie")

    return nominee_dict

In [12]:
best_pic_dict = df.to_dict('r')
best_pic_nominee_df = pd.DataFrame(search_nom_by_name_year(best_pic_dict))
best_pic_nominee_df

/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,tmdb_id,year_film,year_ceremony,oscar_win,no_oscar_nominations,film,oscar_category,nominee,genre_id,tmdb_vote_average,tmdb_vote_count
0,600354,2020,2021,False,0,The Father,Best Picture,"David Parfitt, Jean-Louis Livi, and Philippe C...",[18],83,406
1,583406,2020,2021,False,0,Judas and the Black Messiah,Best Picture,"Shaka King, Charles D. King, and Ryan Coogler","[18, 36]",75,421
2,614560,2020,2021,False,0,Mank,Best Picture,"Ceán Chaffin, Eric Roth, and Douglas Urbanski","[18, 36]",70,820
3,615643,2020,2021,False,0,Minari,Best Picture,Christina Oh,[18],77,323
4,581734,2020,2021,False,0,Nomadland,Best Picture,"Frances McDormand, Peter Spears, Mollye Asher,...","[18, 37]",75,513
5,582014,2020,2021,False,0,Promising Young Woman,Best Picture,"Ben Browning, Ashley Fox, Emerald Fennell, and...","[53, 80, 18]",76,761
6,502033,2020,2021,False,0,Sound of Metal,Best Picture,Bert Hamelinck and Sacha Ben Harroche,"[18, 10402]",78,935
7,556984,2020,2021,False,0,The Trial of the Chicago 7,Best Picture,Marc Platt and Stuart M. Besser,"[18, 36]",78,1614


In [23]:
def get_nom_details(dict):
    
    movie = Movie()
    
    nom_detials_dict = {"tmdb_id":[],
                        "imdb_id":[],
                        "year_film":[],
                        "year_ceremony":[], 
                        "film":[],
                        "oscar_win":[],
                        "no_oscar_nominations":[],
                        "oscar_category":[], 
                        "nominee":[], 
                        "genre_id":[],
                        "staring_casts":[],
                        "casts_popularity":[],
                        "director":[],
                        "producers":[],
                        "screenplay":[],
                        "crew_popularity":[],
                        "budget":[],
                        "revenue":[],
                        "runtime":[],
                        "tmdb_vote_average":[], 
                        "tmdb_vote_count":[]};
                        
    for item in dict:
        if item["tmdb_id"] != 0:
            m = movie.details(item["tmdb_id"])
            nom_detials_dict["tmdb_id"].append(item["tmdb_id"])
            nom_detials_dict["imdb_id"].append(m.imdb_id)
            nom_detials_dict["year_film"].append(item["year_film"])
            nom_detials_dict["year_ceremony"].append(item["year_ceremony"])
            nom_detials_dict["film"].append(item["film"])
            nom_detials_dict["oscar_win"].append(item["oscar_win"])
            nom_detials_dict["no_oscar_nominations"].append(item["no_oscar_nominations"])
            nom_detials_dict["oscar_category"].append(item["oscar_category"])
            nom_detials_dict["nominee"].append(item["nominee"])
            nom_detials_dict["genre_id"].append(item["genre_id"])
            nom_detials_dict["staring_casts"].append(get_top_4_casts(m.casts))
            nom_detials_dict["casts_popularity"].append(get_casts_popularity_score(m.casts))
            nom_detials_dict["director"].append(get_crew_position(m.casts, 0))
            nom_detials_dict["producers"].append(get_crew_position(m.casts, 1))
            nom_detials_dict["screenplay"].append(get_crew_position(m.casts, 2))
            nom_detials_dict["crew_popularity"].append(get_crew_score(m.casts))
            nom_detials_dict["budget"].append(m.budget)
            nom_detials_dict["revenue"].append(m.revenue)
            nom_detials_dict["runtime"].append(m.runtime)
            nom_detials_dict["tmdb_vote_average"].append(item["tmdb_vote_average"])
            nom_detials_dict["tmdb_vote_count"].append(item["tmdb_vote_count"])
                          
    return nom_detials_dict


In [24]:
nom_dict = best_pic_nominee_df.to_dict('r')
best_pic_nominee_df = pd.DataFrame(get_nom_details(nom_dict))
best_pic_nominee_df

,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_win,no_oscar_nominations,oscar_category,nominee,genre_id,...,casts_popularity,director,producers,screenplay,crew_popularity,budget,revenue,runtime,tmdb_vote_average,tmdb_vote_count
0,600354,tt10272386,2020,2021,The Father,False,0,Best Picture,"David Parfitt, Jean-Louis Livi, and Philippe C...",[18],...,58.846,Florian Zeller,"[David Parfitt, Philippe Carcassonne, Jean-Lou...","[Christopher Hampton, Florian Zeller]",22.383,0,924803,97,83,406
1,583406,tt9784798,2020,2021,Judas and the Black Messiah,False,0,Best Picture,"Shaka King, Charles D. King, and Ryan Coogler","[18, 36]",...,66.215,Shaka King,"[Ryan Coogler, Shaka King, Charles D. King]","[Shaka King, Will Berson]",32.282,26000000,4181000,125,75,421
2,614560,tt10618286,2020,2021,Mank,False,0,Best Picture,"Ceán Chaffin, Eric Roth, and Douglas Urbanski","[18, 36]",...,144.436,David Fincher,"[Eric Roth, Ceán Chaffin, Douglas Urbanski]",[Jack Fincher],36.524,20000000,100000,133,70,820
3,615643,tt10633456,2020,2021,Minari,False,0,Best Picture,Christina Oh,[18],...,44.220,Lee Isaac Chung,"[Dede Gardner, Jeremy Kleiner, Christina Oh]",[],126.026,2000000,485158,115,77,323
4,581734,tt9770150,2020,2021,Nomadland,False,0,Best Picture,"Frances McDormand, Peter Spears, Mollye Asher,...","[18, 37]",...,46.505,Chloé Zhao,"[Frances McDormand, Peter Spears, Dan Janvey, ...",[Chloé Zhao],72.538,5000000,0,108,75,513
5,582014,tt9620292,2020,2021,Promising Young Woman,False,0,Best Picture,"Ben Browning, Ashley Fox, Emerald Fennell, and...","[53, 80, 18]",...,95.595,Emerald Fennell,"[Margot Robbie, Ben Browning, Emerald Fennell,...",[],155.232,0,2707180,114,76,761
6,502033,tt5363618,2020,2021,Sound of Metal,False,0,Best Picture,Bert Hamelinck and Sacha Ben Harroche,"[18, 10402]",...,45.078,Darius Marder,"[Bert Hamelinck, Bill Benz, Sacha Ben Harroche...","[Darius Marder, Abraham Marder]",86.194,0,5595,121,78,935
7,556984,tt1070874,2020,2021,The Trial of the Chicago 7,False,0,Best Picture,Marc Platt and Stuart M. Besser,"[18, 36]",...,143.464,Aaron Sorkin,"[Marc Platt, Tyler Thompson, Matt Jackson, Stu...",[Aaron Sorkin],32.899,35000000,107423,130,78,1614


In [25]:
imdb = best_pic_nominee_df.to_dict('r')
omdb_df = pd.DataFrame(get_omdb_nominated(imdb))
omdb_df

/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,imdb_id,oscar_category,nominee,imdb_rating,imdb_votes,revenue,rotten_tomatoes_rating,metascore,total_wins,no_oscar_nominations,total_nominations
0,tt10272386,Best Picture,"David Parfitt, Jean-Louis Livi, and Philippe C...",83,33543,924803,88,88,24,6,127
1,tt9784798,Best Picture,"Shaka King, Charles D. King, and Ryan Coogler",76,27490,4181000,85,85,24,6,51
2,tt10618286,Best Picture,"Ceán Chaffin, Eric Roth, and Douglas Urbanski",70,53366,100000,79,79,47,10,255
3,tt10633456,Best Picture,Christina Oh,76,30604,485158,89,89,101,6,209
4,tt9770150,Best Picture,"Frances McDormand, Peter Spears, Mollye Asher,...",75,54847,2143000,93,93,216,6,140
5,tt9620292,Best Picture,"Ben Browning, Ashley Fox, Emerald Fennell, and...",75,72057,2707180,73,73,101,5,167
6,tt5363618,Best Picture,Bert Hamelinck and Sacha Ben Harroche,78,58498,5595,97,82,74,6,159
7,tt1070874,Best Picture,Marc Platt and Stuart M. Besser,78,127702,107423,76,76,49,6,182


In [26]:
best_pic_findal = pd.merge(best_pic_nominee_df, omdb_df, how='left', on=["imdb_id", "oscar_category", "nominee"])
best_pic_findal.drop(columns= ['revenue_x','no_oscar_nominations_x'],inplace = True)
best_pic_findal = best_pic_findal.rename({'no_oscar_nominations_y':'no_oscar_nominations', 'revenue_y':'revenue'}, axis=1)
best_pic_findal

,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_win,oscar_category,nominee,genre_id,staring_casts,...,tmdb_vote_average,tmdb_vote_count,imdb_rating,imdb_votes,revenue,rotten_tomatoes_rating,metascore,total_wins,no_oscar_nominations,total_nominations
0,600354,tt10272386,2020,2021,The Father,False,Best Picture,"David Parfitt, Jean-Louis Livi, and Philippe C...",[18],"[Anthony Hopkins, Olivia Colman, Mark Gatiss, ...",...,83,406,83,33543,924803,88,88,24,6,127
1,583406,tt9784798,2020,2021,Judas and the Black Messiah,False,Best Picture,"Shaka King, Charles D. King, and Ryan Coogler","[18, 36]","[Daniel Kaluuya, Lakeith Stanfield, Jesse Plem...",...,75,421,76,27490,4181000,85,85,24,6,51
2,614560,tt10618286,2020,2021,Mank,False,Best Picture,"Ceán Chaffin, Eric Roth, and Douglas Urbanski","[18, 36]","[Gary Oldman, Amanda Seyfried, Lily Collins, T...",...,70,820,70,53366,100000,79,79,47,10,255
3,615643,tt10633456,2020,2021,Minari,False,Best Picture,Christina Oh,[18],"[Steven Yeun, Han Yeri, Youn Yuh-jung, Will Pa...",...,77,323,76,30604,485158,89,89,101,6,209
4,581734,tt9770150,2020,2021,Nomadland,False,Best Picture,"Frances McDormand, Peter Spears, Mollye Asher,...","[18, 37]","[Frances McDormand, David Strathairn, Linda Ma...",...,75,513,75,54847,2143000,93,93,216,6,140
5,582014,tt9620292,2020,2021,Promising Young Woman,False,Best Picture,"Ben Browning, Ashley Fox, Emerald Fennell, and...","[53, 80, 18]","[Carey Mulligan, Bo Burnham, Alison Brie, Clan...",...,76,761,75,72057,2707180,73,73,101,5,167
6,502033,tt5363618,2020,2021,Sound of Metal,False,Best Picture,Bert Hamelinck and Sacha Ben Harroche,"[18, 10402]","[Riz Ahmed, Olivia Cooke, Paul Raci, Lauren Ri...",...,78,935,78,58498,5595,97,82,74,6,159
7,556984,tt1070874,2020,2021,The Trial of the Chicago 7,False,Best Picture,Marc Platt and Stuart M. Besser,"[18, 36]","[Yahya Abdul-Mateen II, Sacha Baron Cohen, Jos...",...,78,1614,78,127702,107423,76,76,49,6,182


In [27]:
db_path='Best_Picture_nominee.csv'
best_pic_findal.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: Best_Picture_nominee.csv


### Best Director

In [28]:
wiki_url = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Director'
table_name = 'wikitable' 
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')
soup_table = soup.find_all('table', {'class':table_name})
print(len(soup_table))
df = pd.read_html(str(soup_table))
df = df[11]
category = ['Best Director', 'Best Director', 'Best Director', 'Best Director', 'Best Director']
df['oscar_category'] = category
df.Year = 2020
df['year_ceremony'] = 2021
df['no_oscar_nominations'] = 0;
df['oscar_win'] = False;
df = df.rename({'Director(s)':'nominee', 'Year':'year_film', 'Film':'film'}, axis=1)
df = df.drop(['Ref.'], axis=1)
best_dir_dict = df.to_dict('r')
best_dir_dict_df = pd.DataFrame(search_nom_by_name_year(best_dir_dict))
nom_dict = best_dir_dict_df.to_dict('r')
best_dir_nominee_df = pd.DataFrame(get_nom_details(nom_dict))
imdb = best_dir_nominee_df.to_dict('r')
omdb_df = pd.DataFrame(get_omdb_nominated(imdb))
best_dir_final = pd.merge(best_dir_nominee_df, omdb_df, how='left', on=["imdb_id", "oscar_category", "nominee"])
best_dir_final.drop(columns= ['revenue_x','no_oscar_nominations_x'],inplace = True)
best_dir_final = best_dir_final.rename({'no_oscar_nominations_y':'no_oscar_nominations', 'revenue_y':'revenue'}, axis=1)
best_dir_final

15


/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_win,oscar_category,nominee,genre_id,staring_casts,...,tmdb_vote_average,tmdb_vote_count,imdb_rating,imdb_votes,revenue,rotten_tomatoes_rating,metascore,total_wins,no_oscar_nominations,total_nominations
0,615643,tt10633456,2020,2021,Minari,False,Best Director,Lee Isaac Chung,[18],"[Steven Yeun, Han Yeri, Youn Yuh-jung, Will Pa...",...,77,323,76,30604,485158,89,89,101,6,209
1,582014,tt9620292,2020,2021,Promising Young Woman,False,Best Director,Emerald Fennell,"[53, 80, 18]","[Carey Mulligan, Bo Burnham, Alison Brie, Clan...",...,76,761,75,72057,2707180,73,73,101,5,167
2,614560,tt10618286,2020,2021,Mank,False,Best Director,David Fincher,"[18, 36]","[Gary Oldman, Amanda Seyfried, Lily Collins, T...",...,70,820,70,53366,100000,79,79,47,10,255
3,580175,tt10288566,2020,2021,Another Round,False,Best Director,Thomas Vinterberg,"[35, 18]","[Mads Mikkelsen, Thomas Bo Larsen, Lars Ranthe...",...,77,560,78,60114,3668749,80,80,36,2,55
4,581734,tt9770150,2020,2021,Nomadland,False,Best Director,Chloé Zhao,"[18, 37]","[Frances McDormand, David Strathairn, Linda Ma...",...,75,513,75,54847,2143000,93,93,216,6,140


In [29]:
db_path='Best_Director_nominee.csv'
best_dir_final.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: Best_Director_nominee.csv


### Best Actor

In [30]:
wiki_url = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actor'
table_name = 'wikitable' 
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')
soup_table = soup.find_all('table', {'class':table_name})
print(len(soup_table))
df = pd.read_html(str(soup_table))
df = df[11]
category = ['Best Actor', 'Best Actor', 'Best Actor', 'Best Actor', 'Best Actor']
df['oscar_category'] = category
df.Year = 2020
df['year_ceremony'] = 2021
df['no_oscar_nominations'] = 0;
df['oscar_win'] = False;
df = df.rename({'Actor':'nominee', 'Year':'year_film', 'Film':'film'}, axis=1)
df = df.drop(['Ref.', 'Role(s)'], axis=1)
best_actor_dict = df.to_dict('r')
best_actor_dict_df = pd.DataFrame(search_nom_by_name_year(best_actor_dict))
best_actor_dict = best_actor_dict_df.to_dict('r')
best_actor_nominee_df = pd.DataFrame(get_nom_details(best_actor_dict))
imdb = best_actor_nominee_df.to_dict('r')
omdb_df = pd.DataFrame(get_omdb_nominated(imdb))
best_actor_final = pd.merge(best_actor_nominee_df, omdb_df, how='left', on=["imdb_id", "oscar_category", "nominee"])
best_actor_final.drop(columns= ['revenue_x','no_oscar_nominations_x'],inplace = True)
best_actor_final = best_actor_final.rename({'no_oscar_nominations_y':'no_oscar_nominations', 'revenue_y':'revenue'}, axis=1)
best_actor_final

16


/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


not found


,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_win,oscar_category,nominee,genre_id,staring_casts,...,tmdb_vote_average,tmdb_vote_count,imdb_rating,imdb_votes,revenue,rotten_tomatoes_rating,metascore,total_wins,no_oscar_nominations,total_nominations
0,502033,tt5363618,2020,2021,Sound of Metal,False,Best Actor,Riz Ahmed,"[18, 10402]","[Riz Ahmed, Olivia Cooke, Paul Raci, Lauren Ri...",...,78,935,78,58498,5595,97,82,74,6,159
1,615667,tt10514222,2020,2021,Ma Rainey's Black Bottom,False,Best Actor,Chadwick Boseman (posthumous),"[18, 10402]","[Viola Davis, Chadwick Boseman, Glynn Turman, ...",...,68,592,70,35817,0,87,87,74,5,182
2,600354,tt10272386,2020,2021,The Father,False,Best Actor,Anthony Hopkins,[18],"[Anthony Hopkins, Olivia Colman, Mark Gatiss, ...",...,83,406,83,33543,924803,88,88,24,6,127
3,614560,tt10618286,2020,2021,Mank,False,Best Actor,Gary Oldman,"[18, 36]","[Gary Oldman, Amanda Seyfried, Lily Collins, T...",...,70,820,70,53366,100000,79,79,47,10,255
4,615643,tt10633456,2020,2021,Minari,False,Best Actor,Steven Yeun,[18],"[Steven Yeun, Han Yeri, Youn Yuh-jung, Will Pa...",...,77,323,76,30604,485158,89,89,101,6,209


In [31]:
db_path='Best_Actor_nominee.csv'
best_actor_final.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: Best_Actor_nominee.csv


### Best Actress

In [32]:
wiki_url = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Actress'
table_name = 'wikitable' 
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')
soup_table = soup.find_all('table', {'class':table_name})
print(len(soup_table))
df = pd.read_html(str(soup_table))
df = df[11]
category = ['Best Actress', 'Best Actress', 'Best Actress', 'Best Actress', 'Best Actress']
df['no_oscar_nominations'] = 0;
df['oscar_win'] = False;
df['oscar_category'] = category
df.Year = 2020
df['year_ceremony'] = 2021
df = df.rename({'Actress':'nominee', 'Year':'year_film', 'Film':'film'}, axis=1)
df = df.drop(['Ref.', 'Role(s)'], axis=1)
nom_dict = df.to_dict('r')
nom_df = pd.DataFrame(search_nom_by_name_year(nom_dict))
nom_dict = nom_df.to_dict('r')
nom_df = pd.DataFrame(get_nom_details(nom_dict))
imdb = nom_df.to_dict('r')
omdb_df = pd.DataFrame(get_omdb_nominated(imdb))
best_actress_final = pd.merge(nom_df, omdb_df, how='left', on=["imdb_id", "oscar_category", "nominee"])
best_actress_final.drop(columns= ['revenue_x','no_oscar_nominations_x'],inplace = True)
best_actress_final = best_actress_final.rename({'no_oscar_nominations_y':'no_oscar_nominations', 'revenue_y':'revenue'}, axis=1)
best_actress_final


17


/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


not found
not found
not found


,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_win,oscar_category,nominee,genre_id,staring_casts,...,tmdb_vote_average,tmdb_vote_count,imdb_rating,imdb_votes,revenue,rotten_tomatoes_rating,metascore,total_wins,no_oscar_nominations,total_nominations
0,615667,tt10514222,2020,2021,Ma Rainey's Black Bottom,False,Best Actress,Viola Davis,"[18, 10402]","[Viola Davis, Chadwick Boseman, Glynn Turman, ...",...,68,592,70,35817,0,87,87,74,5,182
1,566076,tt8521718,2020,2021,The United States vs. Billie Holiday,False,Best Actress,Andra Day,"[10402, 18, 36]","[Andra Day, Trevante Rhodes, Garrett Hedlund, ...",...,67,40,63,4991,0,52,52,5,5,19
2,641662,tt11161474,2020,2021,Pieces of a Woman,False,Best Actress,Vanessa Kirby,[18],"[Vanessa Kirby, Shia LaBeouf, Ellen Burstyn, I...",...,71,652,71,35349,0,66,66,5,5,58
3,581734,tt9770150,2020,2021,Nomadland,False,Best Actress,Frances McDormand,"[18, 37]","[Frances McDormand, David Strathairn, Linda Ma...",...,75,513,75,54847,2143000,93,93,216,6,140
4,582014,tt9620292,2020,2021,Promising Young Woman,False,Best Actress,Carey Mulligan,"[53, 80, 18]","[Carey Mulligan, Bo Burnham, Alison Brie, Clan...",...,76,761,75,72057,2707180,73,73,101,5,167


In [33]:
db_path='Best_Actress_nominee.csv'
best_actress_final.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: Best_Actress_nominee.csv


### Best Original Screenplay

In [35]:
wiki_url = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Original_Screenplay'
table_name = 'wikitable' 
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')
soup_table = soup.find_all('table', {'class':table_name})
print(len(soup_table))
df = pd.read_html(str(soup_table))
df = df[8]
category = ['Best Original Screenplay', 'Best Original Screenplay', 'Best Original Screenplay', 'Best Original Screenplay', 'Best Original Screenplay']
df['oscar_category'] = category
df.Year = 2020
df['year_ceremony'] = 2021
df['no_oscar_nominations'] = 0;
df['oscar_win'] = False;
df = df.rename({'Nominees':'nominee', 'Year':'year_film', 'Film':'film'}, axis=1)
nom_dict = df.to_dict('r')
nom_df = pd.DataFrame(search_nom_by_name_year(nom_dict))
nom_dict = nom_df.to_dict('r')
nom_df = pd.DataFrame(get_nom_details(nom_dict))
imdb = nom_df.to_dict('r')
omdb_df = pd.DataFrame(get_omdb_nominated(imdb))
best_org_final = pd.merge(nom_df, omdb_df, how='left', on=["imdb_id", "oscar_category", "nominee"])
best_org_final.drop(columns= ['revenue_x','no_oscar_nominations_x'],inplace = True)
best_org_final = best_org_final.rename({'no_oscar_nominations_y':'no_oscar_nominations', 'revenue_y':'revenue'}, axis=1)
best_org_final

16


/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_win,oscar_category,nominee,genre_id,staring_casts,...,tmdb_vote_average,tmdb_vote_count,imdb_rating,imdb_votes,revenue,rotten_tomatoes_rating,metascore,total_wins,no_oscar_nominations,total_nominations
0,583406,tt9784798,2020,2021,Judas and the Black Messiah,False,Best Original Screenplay,Screenplay: Will Berson & Shaka King; Story: B...,"[18, 36]","[Daniel Kaluuya, Lakeith Stanfield, Jesse Plem...",...,75,421,76,27490,4181000,85,85,24,6,51
1,615643,tt10633456,2020,2021,Minari,False,Best Original Screenplay,Lee Isaac Chung,[18],"[Steven Yeun, Han Yeri, Youn Yuh-jung, Will Pa...",...,77,323,76,30604,485158,89,89,101,6,209
2,582014,tt9620292,2020,2021,Promising Young Woman,False,Best Original Screenplay,Emerald Fennell,"[53, 80, 18]","[Carey Mulligan, Bo Burnham, Alison Brie, Clan...",...,76,761,75,72057,2707180,73,73,101,5,167
3,502033,tt5363618,2020,2021,Sound of Metal,False,Best Original Screenplay,Screenplay: Abraham & Darius Marder; Story: De...,"[18, 10402]","[Riz Ahmed, Olivia Cooke, Paul Raci, Lauren Ri...",...,78,935,78,58498,5595,97,82,74,6,159
4,556984,tt1070874,2020,2021,The Trial of the Chicago 7,False,Best Original Screenplay,Aaron Sorkin,"[18, 36]","[Yahya Abdul-Mateen II, Sacha Baron Cohen, Jos...",...,78,1614,78,127702,107423,76,76,49,6,182


In [36]:
db_path='Best_Org_Screenplay_nominee.csv'
best_org_final.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: Best_Org_Screenplay_nominee.csv


### Best Adapted Screenplay

In [37]:
wiki_url = 'https://en.wikipedia.org/wiki/Academy_Award_for_Best_Adapted_Screenplay'
table_name = 'wikitable' 
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')
soup_table = soup.find_all('table', {'class':table_name})
print(len(soup_table))
df = pd.read_html(str(soup_table))
df = df[10]
df = df.drop('Source Material', axis=1)
category = ['Best Adapted Screenplay', 'Best Adapted Screenplay', 'Best Adapted Screenplay', 'Best Adapted Screenplay', 'Best Adapted Screenplay']
df['oscar_category'] = category
df.Year = 2020
df['year_ceremony'] = 2021
df['no_oscar_nominations'] = 0;
df['oscar_win'] = False;
df = df.rename({'Nominees':'nominee', 'Year':'year_film', 'Film':'film'}, axis=1)
nom_dict = df.to_dict('r')
nom_df = pd.DataFrame(search_nom_by_name_year(nom_dict))
nom_dict = nom_df.to_dict('r')
nom_df = pd.DataFrame(get_nom_details(nom_dict))
imdb = nom_df.to_dict('r')
omdb_df = pd.DataFrame(get_omdb_nominated(imdb))
best_adap_final = pd.merge(nom_df, omdb_df, how='left', on=["imdb_id", "oscar_category", "nominee"])
best_adap_final.drop(columns= ['revenue_x','no_oscar_nominations_x'],inplace = True)
best_adap_final = best_adap_final.rename({'no_oscar_nominations_y':'no_oscar_nominations', 'revenue_y':'revenue'}, axis=1)
best_adap_final

12


/Users/ryu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


not found
not found
not found


,tmdb_id,imdb_id,year_film,year_ceremony,film,oscar_win,oscar_category,nominee,genre_id,staring_casts,...,tmdb_vote_average,tmdb_vote_count,imdb_rating,imdb_votes,revenue,rotten_tomatoes_rating,metascore,total_wins,no_oscar_nominations,total_nominations
0,740985,tt13143964,2020,2021,Borat Subsequent Moviefilm,False,Best Adapted Screenplay,"Screenplay: Sacha Baron Cohen, Peter Baynham, ...",[35],"[Sacha Baron Cohen, Maria Bakalova, Tom Hanks,...",...,65,1498,67,118249,0,68,68,35,2,58
1,600354,tt10272386,2020,2021,The Father,False,Best Adapted Screenplay,Christopher Hampton & Florian Zeller,[18],"[Anthony Hopkins, Olivia Colman, Mark Gatiss, ...",...,83,406,83,33543,924803,88,88,24,6,127
2,581734,tt9770150,2020,2021,Nomadland,False,Best Adapted Screenplay,Chloé Zhao,"[18, 37]","[Frances McDormand, David Strathairn, Linda Ma...",...,75,513,75,54847,2143000,93,93,216,6,140
3,661914,tt10612922,2020,2021,One Night in Miami...,False,Best Adapted Screenplay,Kemp Powers,[18],"[Kingsley Ben-Adir, Eli Goree, Aldis Hodge, Le...",...,72,356,72,19964,0,83,83,54,3,179
4,628534,tt6571548,2020,2021,The White Tiger,False,Best Adapted Screenplay,Ramin Bahrani,"[80, 18]","[Adarsh Gourav, Rajkummar Rao, Priyanka Chopra...",...,70,451,71,43443,0,76,76,54,3,5


In [38]:
db_path='Best_Adap_Screenplay_nominee.csv'
best_adap_final.to_csv(db_path)
print("Exported to local folder:",db_path)

Exported to local folder: Best_Adap_Screenplay_nominee.csv


# References
https://www.kaggle.com/unanimad/the-oscar-award
<br>http://www.omdbapi.com/
<br>https://developers.themoviedb.org/3
<br>https://pypi.org/project/tmdbv3api/